# Basic Tools and Agents

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [4]:
import datetime 

def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M:%S %p")  # Format time in H:MM AM/PM format

In [10]:
get_current_time()

'10:37:21 AM'

In [27]:
from langchain_core.tools import StructuredTool

time_tool = StructuredTool.from_function(
    name="Time",  # Name of the tool
    func=get_current_time,  # Function that the tool will execute
    # Description of the tool
    description="Useful for when you need to know the current time",
)

tools = [time_tool]

Create model

In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

response = model.invoke([('human', 'Hi, what time is it?')])

response.content

"I'm sorry, I am an AI assistant and I do not have the capability to tell time."

In [15]:
model_with_tools = model.bind_tools(tools)

In [17]:
response = model_with_tools.invoke([('human', 'Hi, what time is it?')])

print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")

Content: 
Tool Calls: [{'name': 'Time', 'args': {'config': {}}, 'id': 'call_BZa8EQC963ienVmMVduFjgEY', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create an Agent

In [20]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [21]:
response = agent_executor.invoke({"messages": [('human', 'Hi, what time is it?')]})

response["messages"]

[HumanMessage(content='Hi, what time is it?', id='c1cceb03-f0b5-4c59-93f4-01da33bf19c9'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zKUQGU3gO3G8GhBhd0vEM6JM', 'function': {'arguments': '{"config":{}}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 99, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-59455eec-0c1a-4282-8b97-4f3b652b257b-0', tool_calls=[{'name': 'Time', 'args': {'config': {}}, 'id': 'call_zKUQGU3gO3G8GhBhd0vEM6JM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 99, 'output_tokens': 13, 'total_tokens': 112}),
 ToolMessage(content='10:47:16 AM', name='Time', id='43806b80-81c3-4c94-9b10-df7b25e8ae9f', tool_call_id='call_zKUQGU3gO3G8GhBhd0vEM6JM'),
 AIMessage(content='It is currently 10:47:16 AM.', additional_kwargs={'refusal': None}, response_metadata={'token

In [22]:
for chunk in agent_executor.stream(
    {"messages": [("human", "whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GS7wsERJwRkibZRsvufTU7Rf', 'function': {'arguments': '{"config":{}}', 'name': 'Time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 99, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0e56ea17-888c-4595-bd96-35de258815b4-0', tool_calls=[{'name': 'Time', 'args': {'config': {}}, 'id': 'call_GS7wsERJwRkibZRsvufTU7Rf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 99, 'output_tokens': 13, 'total_tokens': 112})]}}
----
{'tools': {'messages': [ToolMessage(content='10:50:08 AM', name='Time', tool_call_id='call_GS7wsERJwRkibZRsvufTU7Rf')]}}
----
{'agent': {'messages': [AIMessage(content='I currently do not have the ability to check the weather. Please let me know if you would like me to assist you with anything else.', additiona

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).  

In [23]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [24]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
for chunk in agent_executor.stream(
    {"messages": [('human', 'hi')]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 93, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0682e2b6-cef1-400f-b4a5-dcaca93ac632-0', usage_metadata={'input_tokens': 93, 'output_tokens': 10, 'total_tokens': 103})]}}
----
